In [48]:
import pandas as pd
import numpy as np
import datetime

In [49]:
regular_season = pd.read_csv('../data/regular_season.csv')
playoffs = pd.read_csv('../data/playoffs.csv')

In [50]:
# Creating round and game columns for teh regular season so it can be concated with playoffs
regular_season['round'] = 999
regular_season['game'] = 999

# Converting date to a datetime
regular_season['date'] = pd.to_datetime(regular_season['date'])
playoffs['date'] = pd.to_datetime(playoffs['date'])

# Concating the regular season and playoffs together and sorting by date
games = pd.concat([regular_season, playoffs]).sort_values('date')

In [51]:
# Creating a DataFrame to hold the team name and a unique ID
teams = pd.DataFrame({
    'team' : regular_season['home'].sort_values().unique(),
    'ID' : range(100, 100 + len(regular_season['home'].unique()))
    })

# In 2011 the Atlanta Thrashers moved to Winnipeg becoming the Jets. Same team so making the IDs to match.
teams.loc[teams['team'] == 'Winnipeg Jets', 'ID'] = teams.loc[teams['team'] == 'Atlanta Thrashers', 'ID'].values
# In 2014 the Phoenix Coyotes changed their name to the Arizona Coyotes. Making the IDs match
teams.loc[teams['team'] == 'Arizona Coyotes', 'ID'] = teams.loc[teams['team'] == 'Phoenix Coyotes', 'ID'].values

In [52]:
# Merging the team ID onto the regular season data. This ID will be used in the creation of the rating system. The ID is important
# so a team that changes name can still have the correct rating moving into the new season.
games = games.merge(
    right = teams,
    how = 'left',
    left_on = 'home',
    right_on = 'team'
).drop(columns = 'team').rename(columns = {'ID' : 'home_ID'}).merge(
    right = teams,
    how = 'left',
    left_on = 'away',
    right_on = 'team'
).drop(columns = 'team').rename(columns = {'ID' : 'away_ID'})

# Creating column for home and away rating and setting them to the starting value of 1000.
games['home_rating'] = 1000
games['away_rating'] = 1000

In [53]:
# Creating a season rating DataFrame that will be used to calculate the rating carried over to the next season
season_rating = pd.DataFrame({
    'team_ID' : teams['ID'].unique(),
    'season_2000_start' : 1000
})

In [54]:
# List to store DataFrame for each season.
df = []
# Getting a list of seasons.
seasons = games['season'].drop_duplicates().sort_values().to_list()

# Looping through each season.
for season in seasons:
    # Filter for current season.
    games_for_season = games.loc[games['season'] == season].reset_index(drop = True)
    # Setting the intial end of season rating as the start of season rating.
    season_rating[f'season_{season}_end'] = season_rating[f'season_{season}_start']
    
    # Iterating over each game in the current season.
    for index, game in games_for_season.iterrows():
        # Finding the index for the two teams playing
        home_index = game['home_ID'] == season_rating['team_ID']
        away_index = game['away_ID'] == season_rating['team_ID']
        
        # Get home and away ratings for current game.
        home_rating = season_rating.loc[home_index, f'season_{season}_end'].values + 50 # Home ice advantage
        away_rating = season_rating.loc[away_index, f'season_{season}_end'].values
        
        # Setting the home and away rating prior to the game.
        games_for_season.loc[index, 'home_rating'] = home_rating
        games_for_season.loc[index, 'away_rating'] = away_rating

        # Calulating the expected probabilities of home and away team winning.
        if game['round'] < 999:
            rating_diff_home = (away_rating - home_rating) * 1.25
            rating_diff_away = (home_rating - away_rating) * 1.25
        else: 
            rating_diff_home = (away_rating - home_rating)
            rating_diff_away = (home_rating - away_rating)

        E_home = 1 / (1 + 10 ** ((rating_diff_home) / 400))
        E_away = 1 / (1 + 10 ** ((rating_diff_away) / 400))

        # Determining the actual outcome of the game.
        if game['home_goals'] > game['away_goals']:
            S_home, S_away = 1, 0 # Home wins
            auto_correction = 2.05 / ((home_rating - away_rating) * 0.001 + 2.05)
            margin_of_victory = 0.6686 * np.log(abs(game['home_goals'] - game['away_goals'])) + 0.0848
        elif game['home_goals'] < game['away_goals']:
            S_home, S_away = 0, 1 # Away wins
            auto_correction = 2.05 / ((away_rating - home_rating) * 0.001 + 2.05)
            margin_of_victory = 0.6686 * np.log(abs(game['home_goals'] - game['away_goals'])) + 0.0848
        else:
            S_home, S_away = 0.5, 0.5 # Tie
            auto_correction = 1
            margin_of_victory = 1

        # Updating season end rating for home and away.
        season_rating.loc[home_index, f'season_{season}_end'] += 6 * (S_home - E_home) * margin_of_victory * auto_correction
        season_rating.loc[away_index, f'season_{season}_end'] += 6 * (S_away - E_away) * margin_of_victory * auto_correction
    
    # Appending the current season to the list of DataFrame.
    df.append(games_for_season)
    
    # Calculating the starting rating for next season.
    next_season = season + 2 if season == 2003 else season + 1
    season_mean = season_rating[f'season_{season}_end'].mean()
    season_rating[f'season_{next_season}_start'] = season_rating[f'season_{season}_end']  * .7 + season_mean * (1 - .7)
    season_rating[f'season_{next_season}_end'] = season_rating[f'season_{next_season}_start']

# Concat the list of DataFrames.
regular_season = pd.concat(df)


In [55]:
season_rating

,team_ID,season_2000_start,season_2000_end,season_2001_start,season_2001_end,season_2002_start,season_2002_end,season_2003_start,season_2003_end,season_2005_start,...,season_2019_start,season_2019_end,season_2020_start,season_2020_end,season_2021_start,season_2021_end,season_2022_start,season_2022_end,season_2023_start,season_2023_end
0,100,1000,969.081812,978.357268,974.179266,981.925486,997.338398,998.136878,985.305752,989.714026,...,988.141968,966.994547,976.896183,949.131391,964.391974,951.901674,966.331172,904.828661,933.380063,933.380063
1,123,1000,999.674281,999.771997,1015.607099,1010.924969,993.518980,995.463286,964.141738,974.899217,...,976.642405,993.915390,995.740773,983.207273,988.245091,932.190053,952.533037,927.940452,949.558317,949.558317
2,102,1000,953.776618,967.643633,918.524425,942.967097,931.998832,952.399182,949.282611,964.497827,...,1019.310217,1020.167215,1014.117050,1016.373958,1011.461770,1006.653247,1004.657273,1011.157473,1007.810231,1007.810231
3,103,1000,986.868037,990.807626,1008.450558,1005.915391,1002.160150,1001.512105,1011.138785,1007.797149,...,1027.754058,1050.994479,1035.696135,1049.012625,1034.308837,1042.461049,1029.722734,1089.774037,1062.841826,1062.841826
4,104,1000,1021.471229,1015.029860,1018.025845,1012.618092,992.236831,994.565781,994.739305,996.317513,...,952.433949,950.283677,965.198574,939.860384,957.902269,944.969114,961.478380,969.621538,978.735077,978.735077
5,105,1000,979.920482,985.944338,974.912226,982.438558,965.823966,976.076776,1008.315216,1005.820651,...,1014.580840,1006.973029,1004.881120,1000.324089,1000.226862,1044.871219,1031.409854,1032.641137,1022.848796,1022.848796
6,106,1000,994.985648,996.489954,1005.393326,1003.775328,955.984281,969.188997,953.055330,967.138731,...,1007.617461,1023.472285,1016.430600,1037.839039,1026.487327,1059.026436,1041.318506,1052.248675,1036.574072,1036.574072
7,107,1000,974.303098,982.012169,995.132867,996.593007,989.928255,992.949779,953.512148,967.458504,...,990.353404,995.204681,996.643276,979.577900,985.704530,944.740272,961.318190,917.581015,942.306710,942.306710
8,108,1000,1041.168722,1028.818105,1039.559478,1027.691634,1049.547458,1034.683221,1041.092492,1028.764744,...,1005.888566,1026.362241,1018.453569,1043.401691,1030.381184,1059.637202,1041.746042,1053.760968,1037.632678,1037.632678
9,109,1000,980.113607,986.079525,939.616850,957.731795,949.404140,964.582898,945.606330,961.924431,...,1016.695724,1013.240043,1009.268030,975.296729,982.707711,970.479764,979.335835,917.026826,941.918778,941.918778
